# Name and scope
- Specify a name when creating a Variable or Tensor
- Use scope to originaze namescape
    - name_scope 
    - variable_scope
- How to share variables

In [5]:
import tensorflow as tf

### Specify a name when creating a ```Variable``` or ```Tensor```

Variables and tensor in tensorflow have the name attribute that is used to identity them in tensorflow symbolic graph. If you don't specify a name when creating a Variable or tensor, Tensorflow automatically assigns a name for you

In [6]:
tf.reset_default_graph()
a = tf.constant(1)
print(a.name)
b = tf.Variable(1)
print(b.name)

Const:0
Variable:0


You can override the default name by implicitly specifying it

In [7]:
tf.reset_default_graph()
a = tf.constant(1,name='a')
print(a.name)
b = tf.Variable(1,name='b')
print(b.name)

a:0
b:0


In [8]:
print(tf.get_variable_scope())

### Namespace
** Namespace is a way to originaze names for variables and operators in hierarchical manner. Tensorflow introduces two different context managers to alter the name of variables and operators. **

In [11]:
# The first is tf.name_scape.
tf.reset_default_graph()
with tf.name_scope("scope1"):
  a = tf.constant(1, name="a")
  print(a.name)  # prints "scope/a:0"

  b = tf.Variable(1, name="b")
  print(b.name)  # prints "scope/b:0"

  c = tf.get_variable(name="c", shape=[],dtype=tf.int32)
  print(c.name)  # prints "c:0"
with tf.name_scope("scope2"):
  d = a+b+c
tf.summary.FileWriter('namespace',tf.get_default_graph())

scope1/a:0
scope1/b:0
c:0


In [15]:
# The second is tf.variable_scope
tf.reset_default_graph()
with tf.variable_scope("scope"):
  a = tf.constant(1, name="a")
  print(a.name)  # prints "scope/a:0"

  b = tf.Variable(1, name="b")
  print(b.name)  # prints "scope/b:0"

  c = tf.get_variable(name="c", shape=[])
  print(c.name)  # prints "scope/c:0"

scope/a:0
scope/b:0
scope/c:0


### why there are two kinds of scope
In order to implement the variable share, when a variable is defined in some part of your tf program, you can refer to it by ```tf.get_variable``` which can't be influenced by ```tf.name_variable```.

## variables sharing

In [17]:
tf.reset_default_graph()
with tf.variable_scope("scope"):
  a1 = tf.get_variable(name="a", shape=[])
  a2 = tf.get_variable(name="a", shape=[])  # Disallowed

ValueError: Variable scope/a already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-17-aae8e0d5ec5b>", line 3, in <module>
    a1 = tf.get_variable(name="a", shape=[])
  File "/home/yuzhile/work2017/env/tf/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/yuzhile/work2017/env/tf/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2822, in run_ast_nodes
    if self.run_code(code, result):


In [28]:
tf.reset_default_graph()
with tf.variable_scope("scope"):
  a1 = tf.get_variable(name="a", shape=[])
with tf.variable_scope("scope",reuse=True):
  a2 = tf.get_variable(name="a", shape=[])  # Ok
tf.summary.FileWriter('variable_sharing',tf.get_default_graph())

In [26]:
tf.reset_default_graph()
with tf.variable_scope("scope"):
  a = tf.get_variable(name="a", shape=[])
with tf.variable_scope("scope",reuse=True):
  b= tf.get_variable(name="b", shape=[])  # Ok
#f.summary.FileWriter('variable_sharing',tf.get_default_graph())

ValueError: Variable scope/b does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [25]:
tf.reset_default_graph()
image1 = tf.placeholder(dtype=tf.float32,shape=[1,224,224,3])
image2 = tf.placeholder(dtype=tf.float32,shape=[1,224,224,3])
features1 = tf.layers.conv2d(image1, filters=32, kernel_size=3)
#tf.get_variable_scope().reuse_variables()
# Use the same convolution weights to process the second image:
#with tf.variable_scope(tf.get_variable_scope(), reuse=True):
features2 = tf.layers.conv2d(image2, filters=32, kernel_size=3,reuse=True)